In [2]:
import numpy as np

ModuleNotFoundError: No module named 'numpy'

In [ ]:
# initialize weights and bias

def initialize_parameters(layer_dims):

    params = {}  # initialize a dictionary for storing parameters
    L = len(layer_dims)  # number of layers in the network

    for l in range(1, L):
        params['w'+str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) * 0.01